# JOBSHEET 4 : TEKNIK ANALISIS POSE & GEOMETRI TUBUH PADA GAMBAR

## Praktikum D1 — Inisialisasi Kamera dan Akuisisi Citra 
Tujuan:
Mahasiswa mampu menginisialisasi kamera dan menampilkan citra secara real-time sebagai dasar untuk pengolahan visual selanjutnya.
Deskripsi:
Praktikum ini merupakan tahap awal dalam mengenal sistem visi komputer. Mahasiswa akan belajar membuka perangkat kamera, menampilkan citra secara berkelanjutan, serta memastikan bahwa perangkat keras dan perangkat lunak dapat berkomunikasi dengan baik. Proses ini menjadi fondasi bagi semua eksperimen berikutnya yang menggunakan aliran video sebagai input utama.
Langkah:
1.	Jalankan skrip Python menggunakan pustaka opencv-python.
2.	Inisialisasi kamera dengan cv2.VideoCapture(0).
3.	Baca setiap frame dan tampilkan di jendela preview.
4.	Ukur frame per second (FPS) untuk menilai kelancaran akuisisi video.
5.	Tekan tombol q untuk menghentikan proses.
Indikator Keberhasilan:
Citra tampil dengan stabil, FPS berada di atas 10, dan proses dapat dihentikan dengan perintah keluar tanpa error.

In [1]:
import cv2
import time

# Inisialisasi kamera
cap = cv2.VideoCapture(0)

# Periksa apakah kamera bisa dibuka
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka. Coba index 1/2.")

frames, t0 = 0, time.time()

while True:
    ok, frame = cap.read()
    if not ok:
        break

    frames += 1

    # Hitung FPS tiap detik
    if time.time() - t0 >= 1.0:
        cv2.setWindowTitle("Preview", f"Preview (FPS ~ {frames})")
        frames, t0 = 0, time.time()

    # Tampilkan video real-time
    cv2.imshow("Preview", frame)

    # Tekan tombol 'q' untuk keluar
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Bersihkan resource
cap.release()
cv2.destroyAllWindows()

In [5]:
import sys
print(sys.executable)

c:\Users\Dell Inspiron\AppData\Local\Programs\Python\Python313\python.exe


In [1]:
from cvzone.PoseModule import PoseDetector
print("✅ cvzone berhasil diimpor!")

✅ cvzone berhasil diimpor!


## Praktikum D2 — Deteksi Pose dan Analisis Sudut Tubuh

Tujuan:
Mahasiswa mampu menerapkan pose estimation menggunakan MediaPipe dan menghitung sudut sendi tertentu, misalnya sudut lutut.

In [3]:
import cv2
import numpy as np
from cvzone.PoseModule import PoseDetector

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(
    staticMode=False,
    modelComplexity=1,
    enableSegmentation=False,
    detectionCon=0.5,
    trackCon=0.5
)

while True:
    # Tangkap setiap frame dari webcam
    success, img = cap.read()

    # Temukan pose manusia dalam frame
    img = detector.findPose(img)

    # Temukan landmark, bounding box, dan pusat tubuh dalam frame
    # Set draw=True untuk menggambar landmark dan bounding box pada gambar
    lmList, bboxInfo = detector.findPosition(img, draw=True, bboxWithHands=False)

    # Periksa apakah ada landmark tubuh yang terdeteksi
    if lmList:
        # Dapatkan pusat bounding box di sekitar tubuh
        center = bboxInfo["center"]

        # Gambar lingkaran di pusat bounding box
        cv2.circle(img, center, 5, (255, 0, 255), cv2.FILLED)

        # Hitung jarak antara landmark 11 dan 15 dan gambarkan pada gambar
        length, img, info = detector.findDistance(
            lmList[11][0:2],
            lmList[15][0:2],
            img=img,
            color=(255, 0, 0),
            scale=10
        )

        # Hitung sudut antara landmark 11, 13, dan 15 dan gambarkan pada gambar
        angle, img = detector.findAngle(
            lmList[11][0:2],
            lmList[13][0:2],
            lmList[15][0:2],
            img=img,
            color=(0, 0, 255),
            scale=10
        )

        # Periksa apakah sudut mendekati 50 derajat dengan offset 10
        isCloseAngle50 = detector.angleCheck(myAngle=angle, targetAngle=50, offset=10)

        # Cetak hasil pemeriksaan sudut
        print(isCloseAngle50)

    cv2.imshow("Pose + Angle", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

## Praktikum D3 — Deteksi Wajah dan Analisis Kedipan Mata

Tujuan:
Mahasiswa mampu mendeteksi wajah menggunakan MediaPipe Face Mesh dan mengukur perubahan jarak antar landmark mata untuk mendeteksi kedipan.

Deskripsi:
Face Mesh memetakan 468 titik pada wajah, termasuk titik di sekitar mata. Praktikum ini memperkenalkan konsep Eye Aspect Ratio (EAR), yaitu rasio antara jarak vertikal dan horizontal mata yang digunakan untuk menentukan apakah mata dalam kondisi terbuka atau tertutup.
Langkah:

1.	Gunakan modul mediapipe.solutions.face_mesh.
2.	Ambil koordinat beberapa landmark mata (misalnya 33, 133, 145, dan 159).
3.	Hitung Eye Aspect Ratio (EAR):

EAR=dvertikaldhorizontalEAR	=
\frac{d_{vertikal}}{d_{horizontal}}EAR=dhorizontaldvertikal

4.	Tampilkan nilai EAR di layar; nilai menurun saat mata tertutup.
5.	Implementasikan ambang batas untuk mendeteksi kedipan otomatis.

Indikator Keberhasilan:
Sistem mampu menampilkan wajah dengan titik landmark yang stabil dan mendeteksi perubahan nilai EAR secara akurat saat mata berkedip.


In [2]:
import cv2
import numpy as np
from cvzone.FaceMeshModule import FaceMeshDetector

LEFT_EYE = [386, 374, 263, 362]   # top, bottom, left, right
RIGHT_EYE = [159, 145, 133, 33]

def dist(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

cap = cv2.VideoCapture(0)
detector = FaceMeshDetector(maxFaces=1)

blink_count = 0
closed_frames = 0
CLOSED_FRAMES_THRESHOLD = 2
EYE_AR_THRESHOLD = 0.25
is_closed = False

while True:
    success, img = cap.read()
    img, faces = detector.findFaceMesh(img, draw=True)

    if faces:
        face = faces[0]

        # EAR kiri
        vL = dist(face[LEFT_EYE[0]], face[LEFT_EYE[1]])
        hL = dist(face[LEFT_EYE[2]], face[LEFT_EYE[3]])
        earL = vL / hL

        # EAR kanan
        vR = dist(face[RIGHT_EYE[0]], face[RIGHT_EYE[1]])
        hR = dist(face[RIGHT_EYE[2]], face[RIGHT_EYE[3]])
        earR = vR / hR

        ear = (earL + earR) / 2

        cv2.putText(img, f"EAR: {ear:.3f}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)

        if ear < EYE_AR_THRESHOLD:
            closed_frames += 1
            if closed_frames >= CLOSED_FRAMES_THRESHOLD and not is_closed:
                blink_count += 1
                is_closed = True
        else:
            closed_frames = 0
            is_closed = False

        cv2.putText(img, f"Blink: {blink_count}", (20, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,0), 2)

    cv2.imshow("Blink Detection", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


## Praktikum D4 — Deteksi Tangan dan Penghitungan Jumlah Jari

Tujuan:
Mahasiswa mampu mendeteksi tangan manusia dan menghitung jumlah jari yang terangkat menggunakan landmark tangan dari MediaPipe.

Deskripsi:
MediaPipe Hands mendeteksi 21 landmark pada setiap tangan. Berdasarkan hubungan antara ujung jari (tip) dan sendi bawahnya (PIP), dapat ditentukan apakah jari tersebut dalam posisi terangkat atau tidak. Pendekatan ini digunakan untuk menghitung jumlah jari terbuka.

Langkah:

1.	Gunakan mediapipe.solutions.hands untuk memperoleh koordinat landmark.
2.	Bandingkan posisi tip (ujung jari) terhadap PIP (sendi bawah) berdasarkan sumbu vertikal (y).
3.	Tentukan logika sederhana: jika ytip<ypipy_{tip} < y_{pip}ytip<ypip, maka jari dianggap terangkat.
4.	Tampilkan jumlah jari yang terangkat di layar.
5.	Uji sistem dengan berbagai kombinasi jumlah jari.

Indikator Keberhasilan:
Jumlah jari yang terangkat terdeteksi dengan benar (akurasi >80%) pada berbagai kondisi pencahayaan.


In [5]:
import cv2
from cvzone.HandTrackingModule import HandDetector

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = HandDetector(
    staticMode=False,
    maxHands=1,
    modelComplexity=1,
    detectionCon=0.5,
    minTrackCon=0.5
)

while True:
    ok, img = cap.read()
    if not ok:
        break

    hands, img = detector.findHands(img, draw=True, flipType=True)

    if hands:
        hand = hands[0]                     # dict berisi "lmList", "bbox", dll
        fingers = detector.fingersUp(hand)  # list panjang 5 berisi 0/1
        count = sum(fingers)

        cv2.putText(
            img,
            f"Fingers: {count}   {fingers}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            (0, 255, 0),
            2
        )

    cv2.imshow("Hands + Fingers", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Praktikum D5 — Pengenalan Gestur Tangan (Hand Gesture Recognition)

Tujuan:
Mahasiswa mampu mengenali gestur tangan sederhana berdasarkan hubungan geometris antar landmark.

Deskripsi:
Gestur tangan merupakan salah satu bentuk komunikasi nonverbal yang dapat diterjemahkan oleh sistem visi komputer. Dalam praktikum ini, mahasiswa akan merancang aturan geometris untuk mengenali gestur dasar seperti “Thumbs Up”, “OK”, dan “Rock–Paper– Scissors”.

Langkah:

1.	Gunakan landmark kunci seperti ujung ibu jari (4), ujung telunjuk (8), pergelangan tangan (0), dan lainnya.
2.	Definisikan kriteria masing-masing gestur, misalnya:
o	OK: jarak antara ujung ibu jari dan telunjuk < ambang tertentu.
o	Thumbs Up: ibu jari mengarah ke atas dan jauh dari pergelangan.
o	Rock/Paper/Scissors: menggunakan rata-rata jarak ujung jari ke pergelangan.
3.	Tampilkan label gestur yang dikenali pada jendela kamera.
4.	Uji dengan beberapa posisi tangan.

Indikator Keberhasilan:
Sistem dapat mengenali minimal tiga jenis gestur tangan dengan akurasi yang konsisten dan respons waktu <0.3 detik per frame.


In [6]:
import cv2
import numpy as np
from cvzone.HandTrackingModule import HandDetector

def dist(a, b):
    return np.linalg.norm(np.array(a) - np.array(b))

def classify_gesture(hand):
    # hand["lmList"] berisi 21 titik (x,y,z) dalam piksel saat flipType=True
    lm = hand["lmList"]

    wrist = np.array(lm[0][:2])
    thumb_tip = np.array(lm[4][:2])
    index_tip = np.array(lm[8][:2])
    middle_tip = np.array(lm[12][:2])
    ring_tip = np.array(lm[16][:2])
    pinky_tip = np.array(lm[20][:2])

    # Heuristik jarak relatif
    r_mean = np.mean([
        dist(index_tip, wrist),
        dist(middle_tip, wrist),
        dist(ring_tip, wrist),
        dist(pinky_tip, wrist),
        dist(thumb_tip, wrist)
    ])

    # Aturan:
    if dist(thumb_tip, index_tip) < 35:
        return "OK"

    # Thumbs up: ibu jari tinggi (y kecil), jauh dari wrist
    if (thumb_tip[1] < wrist[1] - 40) and (dist(thumb_tip, wrist) > 0.8 * dist(index_tip, wrist)):
        return "THUMBS_UP"

    if r_mean < 120:
        return "ROCK"

    if r_mean > 200:
        return "PAPER"

    if (
        dist(index_tip, wrist) > 180 and
        dist(middle_tip, wrist) > 180 and
        dist(ring_tip, wrist) < 160 and
        dist(pinky_tip, wrist) < 160
    ):
        return "SCISSORS"

    return "UNKNOWN"


cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = HandDetector(
    staticMode=False,
    maxHands=1,
    modelComplexity=1,
    detectionCon=0.5,
    minTrackCon=0.5
)

while True:
    ok, img = cap.read()
    if not ok:
        break

    hands, img = detector.findHands(img, draw=True, flipType=True)

    if hands:
        label = classify_gesture(hands[0])
        cv2.putText(
            img,
            f"Gesture: {label}",
            (20, 40),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.9,
            (0, 255, 255),
            2
        )

    cv2.imshow("Hand Gestures (cvzone)", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## Praktikum D6 — Analisis Gerakan Tubuh dan Penghitung Aktivitas Tujuan:
Mahasiswa mampu membuat sistem penghitung aktivitas fisik (seperti squat dan push-up) menggunakan data landmark pose dan logika kondisi gerakan.

Deskripsi:
Praktikum ini mengintegrasikan kemampuan deteksi pose dengan logika analisis gerakan. Mahasiswa akan menerapkan penghitungan sudut dan rasio jarak untuk mengenali dua posisi utama (naik dan turun), serta menerapkan teknik debounce untuk menghindari penghitungan ganda.

Langkah:

1.	Gunakan MediaPipe Pose untuk mendeteksi titik tubuh yang relevan.
2.	Untuk squat: hitung sudut lutut kiri dan kanan, tentukan kondisi up (θ > 160°) dan
down (θ < 80°).
3.	Untuk push-up: gunakan rasio jarak bahu–pergelangan terhadap bahu–pinggul.
4.	Implementasikan logika transisi down → up sebagai satu repetisi.
5.	Tampilkan jumlah repetisi dan status posisi di layar secara real-time.

Indikator Keberhasilan:
Sistem mampu menghitung jumlah gerakan dengan kesalahan di bawah 10% untuk 10–20 repetisi.

In [7]:
import cv2, numpy as np
from collections import deque
from cvzone.PoseModule import PoseDetector

MODE = "squat"      # tekan 'm' untuk toggle ke pushup
KNEE_DOWN, KNEE_UP = 80, 160      # ambang squat (deg)
DOWN_R, UP_R = 0.85, 1.00         # ambang push-up (rasio)
SAMPLE_OK = 4                     # minimal frame konsisten sebelum ganti state

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    raise RuntimeError("Kamera tidak bisa dibuka.")

detector = PoseDetector(staticMode=False, modelComplexity=1,
                        enableSegmentation=False, detectionCon=0.5,
                        trackCon=0.5)

count, state = 0, "up"
debounce = deque(maxlen=6)

def ratio_pushup(lm):
    sh = np.array(lm[11][1:3])
    wr = np.array(lm[15][1:3])
    hp = np.array(lm[23][1:3])
    return np.linalg.norm(sh - wr) / (np.linalg.norm(sh - hp) + 1e-8)

while True:
    ok, img = cap.read()
    if not ok:
        break

    img = detector.findPose(img, draw=True)
    lmList, _ = detector.findPosition(img, draw=False)

    flag = None

    if lmList:
        if MODE == "squat":
            angL, img = detector.findAngle(
                lmList[23][0:2],
                lmList[25][0:2],
                lmList[27][0:2],
                img=img,
                color=(0, 0, 255),
                scale=10
            )

            angR, img = detector.findAngle(
                lmList[24][0:2],
                lmList[26][0:2],
                lmList[28][0:2],
                img=img,
                color=(0, 255, 0),
                scale=10
            )

            ang = (angL + angR) / 2.0

            if ang < KNEE_DOWN:
                flag = "down"
            elif ang > KNEE_UP:
                flag = "up"

            cv2.putText(img, f"Knee: {ang:5.1f}", (20, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

        else:
            r = ratio_pushup(lmList)

            if r < DOWN_R:
                flag = "down"
            elif r > UP_R:
                flag = "up"

            cv2.putText(img, f"Ratio: {r:4.2f}", (20, 70),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,255), 2)

    debounce.append(flag)

    if debounce.count("down") >= SAMPLE_OK and state == "up":
        state = "down"

    if debounce.count("up") >= SAMPLE_OK and state == "down":
        state = "up"
        count += 1

    cv2.putText(img, f"Mode: {MODE.upper()} Count: {count}", (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,255,255), 2)

    cv2.putText(img, f"State: {state}", (20, 100),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)

    cv2.imshow("Pose Counter", img)
    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    if key == ord('m'):
        MODE = "pushup" if MODE == "squat" else "squat"

cap.release()
cv2.destroyAllWindows()
